In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from emnist import extract_training_samples, extract_test_samples

In [2]:
x_train, y_train = extract_training_samples("letters")
x_test, y_test = extract_test_samples('letters')
img_side = 28
input_shape = (img_side, img_side, 1)
num_classes = 27

In [3]:
x_train = x_train.reshape(-1, img_side, img_side, 1)
x_test = x_test.reshape(-1, img_side, img_side, 1)

ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((124800, 28, 28, 1), (124800,), (20800, 28, 28, 1), (20800,))

In [4]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(x_train.shape[0])
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [5]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential([
    Conv2D(16, (3, 3), padding='same', input_shape=input_shape, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), padding='same', activation= 'relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), padding='same', activation= 'relu'),
    MaxPooling2D(pool_size =(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax'),
])

In [7]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    verbose=1,
    patience=10,
    restore_best_weights=True)

model.fit(
    ds_train,
    epochs=200,
    verbose=1,
    validation_data=ds_test,
    callbacks=[es]
)


Epoch 1/200
975/975 [==============================] - 31s 32ms/step - loss: 0.5969 - accuracy: 0.8166 - val_loss: 0.3163 - val_accuracy: 0.8990
Epoch 2/200
975/975 [==============================] - 31s 32ms/step - loss: 0.2659 - accuracy: 0.9121 - val_loss: 0.2629 - val_accuracy: 0.9140
Epoch 3/200
975/975 [==============================] - 39s 40ms/step - loss: 0.2159 - accuracy: 0.9274 - val_loss: 0.2206 - val_accuracy: 0.9267
Epoch 4/200
975/975 [==============================] - 40s 41ms/step - loss: 0.1893 - accuracy: 0.9355 - val_loss: 0.2173 - val_accuracy: 0.9288
Epoch 5/200
975/975 [==============================] - 38s 39ms/step - loss: 0.1715 - accuracy: 0.9404 - val_loss: 0.1967 - val_accuracy: 0.9332
Epoch 6/200
975/975 [==============================] - 37s 38ms/step - loss: 0.1586 - accuracy: 0.9444 - val_loss: 0.2005 - val_accuracy: 0.9348
Epoch 7/200
975/975 [==============================] - 38s 39ms/step - loss: 0.1451 - accuracy: 0.9482 - val_loss: 0.2090 - val_ac

In [8]:
model.evaluate(ds_test)

163/163 [==============================] - 1s 9ms/step - loss: 0.1893 - accuracy: 0.9382


[0.18927590548992157, 0.9382211565971375]

In [9]:
model.save('mnist_cnn_model.h5')